В этом файле код, который заходит на стену группы Вконтакте и вытаскивает со стены все посты, комменты, комменты к комментам и количество лайков к ним.
Конкретная группа прописывается в параметрах 'domain' и'owner_id'


In [1]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from yaml import load
from pprint import pprint
with open('config.yaml', 'r') as f:
    config = load(f)
token = config['access_token']

In [2]:
# получает вложенные комменты
def get_inner_comments(post_id, comment_id):
    url = 'https://api.vk.com/method/wall.getComments'
    params = {
    'owner_id': '-98492689',
    'post_id': post_id,
    'need_likes': 1,
    'comment_id': comment_id,
    'access_token': token,
    'v': 5.92}
    info = requests.get(url, params = params).json()['response']['items']

    return info

In [3]:
col_names = ['id','text','likes', 'label']

In [4]:
#записывает инфу и по посту и по комментам
def get_comments_info(text_dict, post_id):
    count1 = 100
    offset1 = 0
    flag = True
    while flag == True:
        params = {
        'owner_id': '-98492689',
        'post_id': post_id,
        'need_likes': 1,
        'preview_length': 0,
        'count': count1,
        'offset': offset1,
        'access_token': token,
        'v': 5.92}
        data = requests.get('https://api.vk.com/method/wall.getComments', params = params).json()['response']['items']
        if len(data) == 0:
            flag = False
        for item in data:
            if 'deleted' in item:
                continue
            id_com = item['id']
            text_dict[id_com] = [id_com, item['text'], item['likes']['count'], 'comment']
            if item['thread']['count'] > 0:
                data_lev1 = get_inner_comments(post_id, id_com)
                for el in data_lev1:
                    id_com_lev1=el['id']
                    text_dict[id_com_lev1] = [id_com_lev1, el['text'], el['likes']['count'],  'comment_to_comment']
        offset1 += count1
    return text_dict
# text_dict[post_id] = [post_id, post_text, post_likes, 'post']

In [5]:
text_dict = {}
count = 100
offset = 0
flag_main = True
while flag_main == True:
    params = {
    'domain': 'the4gkz',
    'filter': 'owner',
    'count': count,
    'offset': offset,
    'access_token': token,
    'v': 5.92}
    post100 = requests.get('https://api.vk.com/method/wall.get', params = params).json()['response']['items']
    if len(post100) == 0:
            flag_main = False
    else:
        for post_info in post100:
            post_id = post_info['id']
            post_likes =post_info['likes']['count']
            post_text = post_info['text']
            post_comments = post_info['comments']['count']
            text_dict[post_id] = [post_id, post_text, post_likes, 'post']
            if post_comments != 0:
                get_comments_info(text_dict, post_id)
    offset+=count
text_dict

{76000: [76000,
  '#конкурс@the4gkz \nДля вашего удобства собрали все наши конкурсы в один пост. \nМы разыгрываем билеты на следующие концерты: \n \n• Фестиваль EXIT | 4-7.07 — Сербия \nСсылка на конкурс: vk.cc/9bRbtr \nКупить билеты: vk.cc/9bRcFH \n \n• Bullet for My Valentine | 18-19.04 — Питер/Москва \nСсылка на конкурс: vk.cc/9ewSkY \nКупить билеты: http://clc.to/BFMVMsc | http://clc.to/BFMVSPb \n\n• Ladytron | 01.06 — Москва\nСсылка на конкурс: vk.cc/9fxdGL\nКупить билеты: ladytron2019.ru\n \nУчаствуйте, друзья!',
  17,
  'post'],
 76914: [76914,
  'Уже сегодня ночью (по МСК) начнётся один из самых популярных и крупных фестивалей мира — Coachella. Самое главное — будет трансляция выступлений почти всех артистов прямо на официальном канале фестиваля на YouTube (и никаких проблем с ВПН)!\n\nМы решили попробовать организовать этакий совместный просмотр. Включайте трансляцию и обсуждайте выступления в комментариях. Если мы увидим вашу заинтересованность, то будем делать подобное с дру

In [6]:
len(text_dict)

56200

In [7]:
gk4z_texts=pd.DataFrame.from_dict(text_dict,orient='index', 
                                 columns = col_names)
gk4z_texts = gk4z_texts.reset_index(drop=True)
gk4z_texts.tail(100)

,id,text,likes,label
56100,249,где побухать перед вольтой,134,post
56101,248,почему я живу в россии,155,post
56102,247,на сайте осталось 100 билетов успею ли купить,93,post
56103,246,почему в питере шикари приходят на афтепати а ...,102,post
56104,243,бринг ми зе хоризон кубана 2014 видео с концерта,74,post
56105,242,ай си старс почему болит очко,68,post
56106,241,а мит энд грит будет,105,post
56107,226,как снимать весь концерт на планшет чтобы тебя...,208,post
56108,225,nickelback почему так дорого,86,post
56109,206,когда эминем приедет в россию,68,post


In [20]:
gk4z_texts[gk4z_texts['label']=='post'].count()

id       13734
text     13734
likes    13734
label    13734
dtype: int64

In [10]:
gk4z_texts.to_csv('gk4z_texts.csv', index=False)

In [11]:
# checking:
alpha = pd.read_csv('gk4z_texts.csv')
alpha

,id,text,likes,label
0,76000,#конкурс@the4gkz \r\nДля вашего удобства собра...,17,post
1,76914,Уже сегодня ночью (по МСК) начнётся один из са...,34,post
2,76917,"Ого, Нина Кравиц в лайн-апе😲",0,comment
3,76920,"[id17823823|Анастасия], с новым крутым шоу от ...",0,comment_to_comment
4,76922,Скину расписание на сегодня сюда. Время москов...,0,comment
5,76933,"[id18399319|Anastasia], неправда, в Москве вре...",0,comment_to_comment
6,76929,"Трансляция, конечно, моё почтение. Классно выг...",1,comment
7,76930,"Моё увожение Блэкпинк, которые даже с живыми и...",1,comment
8,76934,"Часть пропустила, конец сета у SG Lewis был кл...",0,comment
9,76935,"сейчас посмотрела лайв Rufus Du Sol, очень бод...",0,comment
